In [1]:
#참고: https://soooprmx.com/archives/6882

# 비동기 코루틴을 호출하는 방법
# 비동기 처리를 쓰는 코드는 다음과 같은 식으로 구성한다.

# 비동기로 처리될 루틴을 코루틴으로 정의한다.
# 런루프를 생성하고
# 런루프에 스케줄링한 다음
# 런루프를 돌려 코루틴이 끝나기를 기다린다.

import asyncio

async def lazy_greet(msg, delay=1):
  print('wiat:', delay)
  await asyncio.sleep(delay)
  print(msg)

loop = asyncio.get_event_loop()
loop.run_until_complete(lazy_greet("hello", 3))
loop.close()

wiat: 3
hello


In [1]:
# 여러 개의 비동기 작업을 스케줄링해보자
# 여러개의 코루틴을 한꺼번에 스케줄링하면 위에서 언급한바와 같이 순서대로 실행되면서 
# 내부에서 await가 등장하면 다른 코루틴으로 제어권이 넘어간다. 
# 다음의 예제는 여러 코루틴을 한꺼번에 스케줄링하고 as_completed를 이용해서 하나씩 그 결과를 얻어 처리하는 가장 기본적인 패턴이다. 

import asyncio
import random

async def lazy_greet(msg, delay=1):
  print(msg, "will be displayed in", delay, "seconds")
  await asyncio.sleep(delay)
  return msg.upper()

async def main():
  messages = ['hello', 'world', 'apple', 'banana', 'cherry']
  fts = [asyncio.ensure_future(lazy_greet(m, random.randrange(1, 5))) for m in messages]
  for f in asyncio.as_completed(fts):
      x = await f
      print(x)
 

loop2 = asyncio.get_event_loop()
loop2.run_until_complete(main())
loop2.close()


hello will be displayed in 1 seconds
world will be displayed in 2 seconds
apple will be displayed in 1 seconds
banana will be displayed in 1 seconds
cherry will be displayed in 3 seconds
HELLO
APPLE
BANANA
WORLD
CHERRY
CHERRY


In [1]:
# 타임아웃 시점까지 대기하기
# concurrent.futures의 Executor.wait() 에 대응하는 것은 asyncio.wait() 이다. 
# 여기에는 Futures의 시퀀스와 타임아웃 혹은 종료 조건이 넘겨지고, (done, pending)의 집합 짝이 리턴된다. 
# 중요한 것은 wait 역시 코루틴 함수이며, 따라서 await를 통해서 결과를 받아야 한다는 점이다. 

import asyncio
import random

async def lazy_greet(msg, delay=1):
    print(f'{msg!r} will be displayed in {delay} seconds.')
    await asyncio.sleep(delay)
    return msg.upper()

async def main():
    messages = 'hello world apple banana cherry'.split()
    cos = [lazy_greet(m, random.randrange(1, 5)) for m in messages]
    (done, pending) = await asyncio.wait(cos, timeout=2)
    if pending:
        print("there is {} tasks not completed".format(len(pending)))
        for f in pending:
            f.cancel()
    for f in done:
        print(await f)

#3.7
#asyncio.run(main())
loop = asyncio.get_event_loop()
loop.run_until_complete(main())
loop.close()

'cherry' will be displayed in 3 seconds.
'apple' will be displayed in 2 seconds.
'banana' will be displayed in 4 seconds.
'hello' will be displayed in 4 seconds.
'world' will be displayed in 4 seconds.
there is 5 tasks not completed


In [1]:
# 결과만 취합하기
# as_completed를 통해서 완료된 코루틴의 결과들을 개별적으로 꺼내는 것 말고 한꺼번에 결과를 취합받는 방법도 있다. 
# 이는 asyncio.gather()를 이용하는데, 이 함수는 Executor.map() 함수와 비슷하다. 차이가 있다면,

# 코루틴 함수이다 (await와 같이 쓰임)
# Future의 시퀀스를 인자로 받는게 아니라, 개별 인자들로 받는다.
# 위 예제에서 asyncio.as_complete()를 쓰던 것을 asyncio.gather()로 변경했다. 
# 결과를 출력하는 방식이 바뀔 뿐만 아니라 가장 늦게 끝나는 작업이 끝나야지 한 번에 종료된다.

import asyncio
import random

async def lazy_greet(msg, delay=1):
    print(f'{msg!r} will be displayed in {delay} seconds.')
    await asyncio.sleep(delay)
    return msg.upper()

async def main2():
    messages = 'hello world apple banana cherry'.split()
    cos = [lazy_greet(m, random.randrange(1, 5)) for m in messages]
    res = await asyncio.gather(*cos)
    print(res)

#3.7
#asyncio.run(main())
loop = asyncio.get_event_loop()
loop.run_until_complete(main2())
loop.close()

'world' will be displayed in 2 seconds.
'hello' will be displayed in 2 seconds.
'apple' will be displayed in 3 seconds.
'banana' will be displayed in 3 seconds.
'cherry' will be displayed in 2 seconds.
['HELLO', 'WORLD', 'APPLE', 'BANANA', 'CHERRY']
